In [1]:
# Reset and Restart 

import sys
import os
import numpy as np
import time
from imp import reload
import threading
import csv
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO

reload(DisplayWindows)

CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
#CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()

ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

%matplotlib qt

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


daq
uniphase
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x0000027C22129488>, 'xy_stage': <L2.XYControl.PycromanagerXY object at 0x0000027C221293C8>, 'objective': <L2.ZControl.PycromanagerZ object at 0x0000027C22129548>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x0000027C221295C8>, 'inlet_z': <L2.ZControl.ArduinoZ object at 0x0000027C221294C8>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x0000027C22129608>, 'filter_wheel': <L2.FilterWheelControl.PycromanagerFilter object at 0x0000027C221298C8>, 'excitation_shutter': <L2.ShutterControl.PycromanagerShutter object at 0x0000027C22129748>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x0000027C221297C8>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x0000027C22129808>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x0000027C22129C48>, 'lysis_laser': <L2.LaserControl.Uniphase object at 0x0000027C22136088>}
OPENING:  mmc1 <L1.Controllers.PycromanagerCont

In [2]:

class ImageSaver:
    
    def __init__(self, system:SystemsBuilder.CESystem, folder_prefix=None, data_folder=None):
        self.system = system
        self.prefix = folder_prefix
        self.data_folder = data_folder
        self.images = []
        self._lock = threading.RLock()
        
    def add_callback(self):
        self.system.camera.add_callback(self.callback, tag='save_img')
        
    def remove_callback(self):
        self.system.camera.remove_callback(tag='save_img')
    
    def callback(self, img, *args, **kwargs):
        with self._lock:
            self.images.append(img)
    
    def save_image(self, image_prefix='img', folder_prefix=None, data_folder=None, unique_folder=True):
        """
        Saves the images into an folder with incrementing image names:
        
        image_prefix: what the image filename should be caleld
        folder_prefix: what the data folder will be called
        data_folder: where the parent_directory resides
        unique_folder: When true, create a new folder everytime the function is called
        
        """
        
        if folder_prefix is None:
            prefix = self.prefix
        if data_folder is None:
            data_folder = self.data_folder
    
        if unique_folder:
            data_dir = FileIO.get_data_folder(prefix, data_folder)
        else:
            data_dir = data_folder
        for image in self.images:
            data_file = FileIO.get_data_filename(image_prefix, data_dir, extension='.tiff')
            io.imsave(data_file, image)
            
    
    

In [3]:
from L4 import Focus
reload(Focus)

<module 'L4.Focus' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L4\\Focus.py'>

In [4]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


In [8]:
ce_system.camera.stop()

True

In [9]:
ce_system.camera.set_binning(1)

In [10]:
ce_system.camera.continuous_snap()

HEYYOO


True

In [115]:
# PARAMETERS
target_x = 871
target_y = 585
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 500
_old_exp=10
_old_bin = 1
_old_channel = 0
bins = 4
conversion =0.0003125
obj_40x = 0.0001607717041800643


def pre_fluoresence(auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    ce_system.camera.set_binning(bins)
    ce_system.camera.set_exposure(exp)
    if auto_shutter:
        ce_system.excitation_shutter.set_auto_on()
    #_old_exp = ce_system.camera.exposure
    #ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence(auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    ce_system.camera.set_binning(_old_bin)
    ce_system.excitation_shutter.set_auto_off()
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)
    ce_system.camera.continuous_snap()

def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

def get_brightest_blobs(image):
    close_size=5
    open_size=5
    watershed_footprint = (5,5)
    # Make sure types are the same
    input_image = img_as_float(img)

    # Filter Image
    filtered_image = filters.median(input_image, behavior='ndimage')

    # Edge Detection
    edge_sobel = filters.sobel(filtered_image)

    # Threshold
    thresh = filters.threshold_otsu(filtered_image)
    binary_otsu = filtered_image > thresh

    # Binary Morphology Operations
    structure_element = morphology.disk(close_size)
    closed_image = morphology.binary_closing(binary_otsu, structure_element)
    structure_element = morphology.disk(open_size)
    opened_image = morphology.binary_opening(closed_image, structure_element)

    # Watershed
    distance = distance_transform_edt(opened_image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
    markers = label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=opened_image)
    
    # Labeling and Sorting
    regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))
    df = pd.DataFrame(regions)
    df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)
    
    return df

def move_to_blob(df):
    if df.shape[0] > 0:
        y=df['centroid-0'][0]*bins
        x=df['centroid-1'][0]*bins

        movex = (x-target_x)*conversion
        movey = (y-target_y)*conversion
        ce_system.xy_stage.set_rel_x(movex)
        ce_system.xy_stage.set_rel_y(-movey)
        
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    ce_system.lysis_laser.laser_fire()
    ce_system.objective.set_rel_z(-0.010)

    while time.time()-start_time < injection_time:
        if time.time()-start_time < 3:
            ce_system.objective.set_rel_z(0.005)
            ce_system.xy_stage.set_rel_x(0.001)
            ce_system.lysis_laser.laser_fire()
        time.sleep(0.35)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)

In [25]:
pre_fluoresence(auto_shutter=True)

In [26]:
img = ce_system.camera.snap()

In [28]:
fig,ax = plt.subplots()
ax.hist(img.flatten())

(array([1.357042e+06, 4.398000e+03, 3.571000e+03, 2.640000e+03,
        2.147000e+03, 2.006000e+03, 2.355000e+03, 1.316000e+03,
        4.730000e+02, 3.080000e+02]),
 array([122. , 208.6, 295.2, 381.8, 468.4, 555. , 641.6, 728.2, 814.8,
        901.4, 988. ]),
 <BarContainer object of 10 artists>)

In [27]:
ax.imshow(img)

In [29]:
post_fluoresence(auto_shutter=True)

HEYYOO


In [30]:
las = ce_system.lysis_laser

In [31]:
las.laser_standby()

In [89]:
las.laser_fire()

In [54]:
import threading
import time
fire_event = threading.Event()
fire_event.set()
def fire_laser():
    while fire_event.is_set():
        las.laser_fire()
        time.sleep(0.5)
        
threading.Thread(target=fire_laser).start()

In [56]:
fire_event.clear()

In [68]:
import L4.image_util as img_util

In [74]:
fs = Focus.FindFocus(ce_system)

In [116]:
old_z = ce_system.objective.read_z()
save_to = r"C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs"
ims = img_util.ImageSaver(ce_system, 'calcein_focus', data_folder=save_to)
pre_fluoresence(auto_shutter=True)
ims.add_callback()
fs.search_fibonacci()
ims.remove_callback()
ims.save_image(folder_prefix='fibonacci')
post_fluoresence()

ce_system.objective.set_z(old_z)
time.sleep(3)



save_to = r"C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs"
ims = img_util.ImageSaver(ce_system, 'calcein_focus', data_folder=save_to)
pre_fluoresence(auto_shutter=True)
ims.add_callback()
fs.search_step_global()
ims.remove_callback()
ims.save_image(folder_prefix='step_global')
post_fluoresence()


Position: 8.837125131683424
Position: 8.847100131832063
Position: 8.853075131921099
Position: 8.843025131771341
Position: 8.841025131741539


KeyboardInterrupt: 

In [81]:
save_to = r"C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs"
ims = img_util.ImageSaver(ce_system, 'calcein_focus', data_folder=save_to)
pre_fluoresence(auto_shutter=True)
ims.add_callback()
fs.search_step_global()
ims.remove_callback()
ims.save_image(folder_prefix='step_global')
post_fluoresence()


meaure: -1, step: -2, pos= 8.748500130362808
meaure: -3, step: -1, pos= 8.755450130466372
meaure: 1, step: 0, pos= 8.762400130569935
meaure: -1, step: 1, pos= 8.76937513067387
meaure: -5, step: 2, pos= 8.776300130777061
meaure: 1, step: 3, pos= 8.783275130880996
meaure: -1, step: 4, pos= 8.790250130984932


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\step_global_00000\img.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\step_global_00000\img_00000.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\step_global_00000\img_00001.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\step_global_00000\img_00002.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 

HEYYOO


In [73]:
ims.save_image(folder_prefix='test')

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\test_00000\img.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\test_00000\img_00000.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\test_00000\img_00001.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\test_00000\img_00002.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\test_00000\

In [137]:
ims = img_util.ImageSaver(ce_system, 'calcein_leaking', data_folder=save_to)
imf = img_util.ImageSaver(ce_system, 'calcein_lysis', data_folder=save_to)
def run():
    dis = 0.01
    
    pre_fluoresence(auto_shutter=True)
    ims.add_callback()
    ce_system.camera.snap()
    #post_fluoresence(auto_shutter=True)
    ce_system.filter_wheel.set_channel(0)
    #imf.add_callback()
    old_z = ce_system.objective.read_z()
    ce_system.objective.set_rel_z(dis)
    time.sleep(0.5)
    las.laser_standby()
    las.laser_fire()
    ce_system.filter_wheel.set_channel(1)

    ce_system.objective.set_z(old_z)
    #time.sleep(1)
    #imf.remove_callback()
    pre_fluoresence(auto_shutter=True)
    #ims.add_callback()
    st = time.time()
    while time.time()-st<15:
        ce_system.camera.snap()
        time.sleep(exp/2000) 
    ims.remove_callback()
    post_fluoresence()

threading.Thread(target=run).start()

HEYYOO


In [136]:
ims.save_image(image_prefix='10um')
#imf.save_image()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_leaking_00011\10um_00000.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_leaking_00011\10um_00001.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_leaking_00011\10um_00002.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_leaking_00011\10um_00003.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke

In [117]:
post_fluoresence()

HEYYOO


In [96]:
las.laser_fire()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_lysis\img.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_lysis\img_00000.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_lysis\img_00001.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: C:\Users\Luke\Documents\Brae\10-3-2020 focus\focus imgs\calcein_lysis\img_00002.tiff is a low contrast image
  io.imsave(data_file, image)
